**Use Discogs’ publicly available data to build an ETL pipeline (in python) that creates a table of music label information.**

Extract data from the monthly XML dump file of all labels into a pandas DataFramex_release_year


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# url
df = pd.read_xml(r"C:\Users\rdog0\OneDrive\Desktop\brianna's stuff\discogs_20230101_labels.xml")

In [4]:
df

,images,id,name,contactinfo,profile,data_quality,urls,sublabels,parentLabel
0,NaN,1,Planet E,Planet E Communications\r\nP.O. Box 27218\r\nD...,[a=Carl Craig]'s classic techno label founded ...,Needs Vote,NaN,NaN,None
1,NaN,2,Earthtones Recordings,Seasons Recordings\r\n2236 Pacific Avenue\r\nS...,California deep house label founded by [a=Jami...,Correct,NaN,NaN,None
2,NaN,3,Seasons Recordings,"Seasons Recordings\r\nCosta Mesa, CA 92627\r\n...",California deep-house label founded by [a=Jami...,Needs Vote,NaN,NaN,None
3,NaN,4,Siesta Music,Siesta Records\r\n1913 Via Encantadoras\r\nSan...,None,Needs Vote,NaN,NaN,None
4,NaN,5,Svek,Svek office \r\nStephan Grieder \r\nFax: +46 (...,None,Correct,NaN,NaN,Goldhead Music
...,...,...,...,...,...,...,...,...,...
1946241,NaN,3078049,5th Gear Productions,None,None,Needs Major Changes,NaN,NaN,None
1946242,NaN,3078052,"Tunnell, Jimi Music",None,None,Needs Major Changes,NaN,NaN,None
1946243,NaN,3078055,Warrior Gospel Records,None,None,Needs Major Changes,NaN,NaN,None
1946244,NaN,3078058,Allegretto Music,None,None,Needs Major Changes,NaN,NaN,None


Step 2: Transform data

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1946246 entries, 0 to 1946245
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   images        float64
 1   id            int64  
 2   name          object 
 3   contactinfo   object 
 4   profile       object 
 5   data_quality  object 
 6   urls          float64
 7   sublabels     float64
 8   parentLabel   object 
dtypes: float64(3), int64(1), object(5)
memory usage: 133.6+ MB


In [11]:
df.isnull().sum()

images          1946246
id                    0
name                  0
contactinfo     1811484
profile         1434545
data_quality          0
urls            1946246
sublabels       1946246
parentLabel     1834877
dtype: int64

In [40]:
# Drop columns
df.drop(columns=['images','sublabels','urls','contactinfo','data_quality'])

,id,name,profile,urls,parentLabel
0,1,Planet E,[a=Carl Craig]'s classic techno label founded ...,NaN,None
1,2,Earthtones Recordings,California deep house label founded by [a=Jami...,NaN,None
2,3,Seasons Recordings,California deep-house label founded by [a=Jami...,NaN,None
3,4,Siesta Music,None,NaN,None
4,5,Svek,None,NaN,Goldhead Music
...,...,...,...,...,...
1946241,3078049,5th Gear Productions,None,NaN,None
1946242,3078052,"Tunnell, Jimi Music",None,NaN,None
1946243,3078055,Warrior Gospel Records,None,NaN,None
1946244,3078058,Allegretto Music,None,NaN,None


In [39]:
# add columns from Discogs API: total releases, min release year, max release year
user_name = 'brianna023'
token = 'pKEStseWqjbnCsLCTinkDPQwvNhcmNDLzdXfyYaH'


# ingest data
def get_labels(label_id, user_name, token):
    # get url
    url = f'https://api.discogs.com/labels/{label_id}/releases'
    # define headers
    headers = {
        'User-Agent':user_name,
        'Authorization':f'Discogs token={token}'}
    # set params
    params = {'per_page': 25}     # set a per page limit
    #define reponse
    response = requests.get(url, headers=headers, params=params)
    # create list for labels
    label_releases = []
    
    if response.status_code != 200:
        raise Exception("Failed to get token from Discogs API")
    return response.json()

# ex: use track 33
label_id = 1

# defind label_releases
label_releases = get_labels(label_id, user_name, token)
label_releases

{'pagination': {'page': 1,
  'pages': 23,
  'per_page': 25,
  'items': 574,
  'urls': {'last': 'https://api.discogs.com/labels/1/releases?per_page=25&page=23',
   'next': 'https://api.discogs.com/labels/1/releases?per_page=25&page=2'}},
 'releases': [{'status': 'Accepted',
   'format': 'CD, Comp, Mixed',
   'catno': '!K7067cd',
   'thumb': 'https://i.discogs.com/V60lNnC3DxsCaNOabHjHbO5Xd6DIp6AnQHZCOPbra8w/rs:fit/g:sm/q:40/h:150/w:150/czM6Ly9kaXNjb2dz/LWRhdGFiYXNlLWlt/YWdlcy9SLTEwMTgt/MTE3MTczMTgzMS5q/cGVn.jpeg',
   'resource_url': 'https://api.discogs.com/releases/1018',
   'title': 'Electro Boogie Vol 2 (The Throwdown)',
   'id': 1018,
   'year': 1998,
   'artist': 'Dave Clarke',
   'stats': {'community': {'in_wantlist': 171, 'in_collection': 662},
    'user': {'in_wantlist': 0, 'in_collection': 0}}},
  {'status': 'Accepted',
   'format': 'CD, Mixed',
   'catno': 'K7071CD, !K7071cd',
   'thumb': 'https://i.discogs.com/AW3gz3RTecljLNZ9mURNxsJbWUUwjr4cgUQzJ32DNb8/rs:fit/g:sm/q:40/h:150/

In [45]:
release_sections = soup.find_all('div', class_='pagination')
release_sections

NameError: name 'soup' is not defined

In [43]:
# Find total releases, min, and max years released
rele
# num_of_releases
# min_release_year 
# max_release_year

# releases = []
# year = []

for release in label_releases:
    title = release['title']
    year = release['year']
print(title,year)

TypeError: string indices must be integers, not 'str'